In [1]:
# Importing Necessary Libraries
# =============================

import pyspark
from pyspark.sql.functions import col
from pyspark.sql import functions as F, SparkSession
from pyspark.sql.types import FloatType

In [2]:
# Reading Training from Tables into DataFrames
# ==========================================================
# Prior to running this code cell, the training data must be uploaded onto
# DataBricks as a table named "training_data".

# Training Data
df = sqlContext.table("training_data")
df.printSchema()

root
-- archived: boolean (nullable = true)
-- author: string (nullable = true)
-- author_cakeday: boolean (nullable = true)
-- author_flair_background_color: string (nullable = true)
-- author_flair_css_class: string (nullable = true)
-- author_flair_richtext: array (nullable = true)
 |-- element: string (containsNull = true)
-- author_flair_text: string (nullable = true)
-- author_flair_text_color: string (nullable = true)
-- author_flair_type: string (nullable = true)
-- brand_safe: boolean (nullable = true)
-- can_gild: boolean (nullable = true)
-- contest_mode: boolean (nullable = true)
-- created_utc: long (nullable = true)
-- distinguished: string (nullable = true)
-- domain: string (nullable = true)
-- edited: boolean (nullable = true)
-- gilded: long (nullable = true)
-- hidden: boolean (nullable = true)
-- hide_score: boolean (nullable = true)
-- id: string (nullable = true)
-- is_crosspostable: boolean (nullable = true)
-- is_reddit_media_domain: boolean (nullable = true)
-- is_self: boolean (nullable = true)
-- is_video: boolean (nullable = true)
-- link_flair_css_class: string (nullable = true)
-- link_flair_richtext: array (nullable = true)
 |-- element: string (containsNull = true)
-- link_flair_text: string (nullable = true)
-- link_flair_text_color: string (nullable = true)
-- link_flair_type: string (nullable = true)
-- locked: boolean (nullable = true)
-- media: string (nullable = true)
-- no_follow: boolean (nullable = true)
-- num_comments: long (nullable = true)
-- num_crossposts: long (nullable = true)
-- over_18: boolean (nullable = true)
-- parent_whitelist_status: string (nullable = true)
-- permalink: string (nullable = true)
-- post_hint: string (nullable = true)
-- preview: struct (nullable = true)
 |-- enabled: boolean (nullable = true)
 |-- images: array (nullable = true)
 | |-- element: struct (containsNull = true)
 | | |-- id: string (nullable = true)
 | | |-- resolutions: array (nullable = true)
 | | | |-- element: struct (containsNull = true)
 | | | | |-- height: long (nullable = true)
 | | | | |-- url: string (nullable = true)
 | | | | |-- width: long (nullable = true)
 | | |-- source: struct (nullable = true)
 | | | |-- height: long (nullable = true)
 | | | |-- url: string (nullable = true)
 | | | |-- width: long (nullable = true)
-- retrieved_on: long (nullable = true)
-- rte_mode: string (nullable = true)
-- score: long (nullable = true)
-- secure_media: string (nullable = true)
-- selftext: string (nullable = true)
-- send_replies: boolean (nullable = true)
-- spoiler: boolean (nullable = true)
-- stickied: boolean (nullable = true)
-- subreddit: string (nullable = true)
-- subreddit_id: string (nullable = true)
-- subreddit_name_prefixed: string (nullable = true)
-- subreddit_type: string (nullable = true)
-- suggested_sort: string (nullable = true)
-- thumbnail: string (nullable = true)
-- thumbnail_height: long (nullable = true)
-- thumbnail_width: long (nullable = true)
-- title: string (nullable = true)
-- url: string (nullable = true)
-- whitelist_status: string (nullable = true)

In [3]:
# Dataframe size
# =============================

row_number = df.count()
column_number = len(df.dtypes)
print('The Train data has ', row_number, 'individual reddits posts (rows) and', column_number, 'associated attributes (columns)')

The Train data has 12525 individual reddits posts (rows) and 58 associated attributes (columns)

In [4]:
# Plot count of each data type.
# =============================

dataname = []
datatype = []
for name, dtype in df.dtypes:
  if dtype not in ['boolean', 'string', 'bigint', 'array<string>']:
    dataname.append(name)
    datatype.append('other')
  else:
    dataname.append(name)
    datatype.append(dtype)
  
datanamedatatype = sqlContext.createDataFrame(zip(dataname, datatype), schema=['dataname', 'datatype'])
# x.show(100)
display(datanamedatatype.groupby('datatype').count())

datatype,count
other,1
boolean,18
string,29
bigint,8
array,2


In [5]:
# # Some data exploration of each column.
# # =============================

# for item in dataname:
#   df.select(item, 'score', 'id').groupby(item).agg({'id':'count', 'score':'average'}).show(5)

In [6]:
# Columns of each data type.
# =============================

datanamedatatype.createOrReplaceTempView("datanamedatatype")

sql_results_string = spark.sql("SELECT dataname FROM datanamedatatype WHERE datatype = 'string'")
sql_results_bigint = spark.sql("SELECT dataname FROM datanamedatatype WHERE datatype = 'bigint'")
sql_results_boolean = spark.sql("SELECT dataname FROM datanamedatatype WHERE datatype = 'boolean'")

string_cols = sql_results_string.select("dataname").rdd.flatMap(lambda x: x).collect()
bigint_cols = sql_results_bigint.select("dataname").rdd.flatMap(lambda x: x).collect()
boolean_cols = sql_results_boolean.select("dataname").rdd.flatMap(lambda x: x).collect()

In [7]:
# Function for finding Columns with Missing Values/Null values.
# =============================

def missingcounts(df):
  dfMissing = df.select([F.count(F.when(F.isnan(i) | \
                                   F.col(i).contains('NA') | \
                                   F.col(i).contains('NULL') | \
                                   F.col(i).isNull(), i)).alias(i) \
                    for i in df.columns])
  return dfMissing

In [8]:
# Null count for String cols.
# =============================

dfstring = df.select([c for c in df.columns if c in string_cols])
dfstringMissing = missingcounts(dfstring)

NullCount = []
for i in string_cols:
  tableList = dfstringMissing.select(i).rdd.flatMap(lambda x: x).collect()
  NullCount.append(tableList[0])

dfstringCountNull = sqlContext.createDataFrame(zip(string_cols, NullCount), schema=['String Columns', 'Null/Missing Count'])

display(dfstringCountNull)

String Columns,Null/Missing Count
author,0
author_flair_background_color,11688
author_flair_css_class,12525
author_flair_text,12525
author_flair_text_color,11688
author_flair_type,837
distinguished,12525
domain,0
id,0
link_flair_css_class,12525


In [9]:
# Null count for Bigint cols.
# =============================

dfbigint = df.select([c for c in df.columns if c in bigint_cols])
dfbigintMissing = missingcounts(dfbigint)

NullCount = []
for i in bigint_cols:
  tableList = dfbigintMissing.select(i).rdd.flatMap(lambda x: x).collect()
  NullCount.append(tableList[0])

dfbigintCountNull = sqlContext.createDataFrame(zip(bigint_cols, NullCount), schema=['Bigint Columns', 'Null/Missing Counts'])

display(dfbigintCountNull)

Bigint Columns,Null/Missing Counts
created_utc,0
gilded,0
num_comments,0
num_crossposts,0
retrieved_on,12319
score,0
thumbnail_height,12519
thumbnail_width,12519


In [10]:
# Determing useful columns intuitively - quick data clean.
# =============================

print("Total columns:", len(dataname))

# keep only relevant string cols with low null values or intuitive string columns.
drop_string_cols = []

keep_str_cols = ["author", "domain", "permalink", "subreddit_id", "suggested_sort", "title", "whitelist_status"]
for stringc in string_cols:
  if stringc not in keep_str_cols:
    drop_string_cols.append(stringc)    
    
for item in drop_string_cols:
  if item in dataname:
    dataname.remove(item)

print('Total columns (after removing null/irrelavent string cols):', len(dataname), ' - dropped ', len(drop_string_cols), ' string out of ', len(string_cols), '(includes permalink, author and title that are used for deriving other features but not used as features themselves)')

# drop all bigint_cols with null values or with same values.
drop_bigint_cols = ['thumbnail_height', 'thumbnail_width', 'retrieved_on']

countr = 0 
for bigintc in bigint_cols:
  x = dfbigint.select(bigintc).distinct().count()
  if x <= 1:
    drop_bigint_cols.append(bigintc)
    countr += 1

for item in drop_bigint_cols:
  if item in dataname:
    dataname.remove(item)
    
print('Total columns (after removing 3 null bigint cols and ', countr,  ' bigint cols with same values):', len(dataname), ' - dropped ', len(drop_bigint_cols), ' bigint out of ', len(bigint_cols), ('includes permalink, author and title that are used for deriving other features but not used as features themselves'))

# drop boolean columns that have same values
drop_boolean_cols = []

# dfboolean = df.select([col(c).cast('double') for c in df.columns if c in boolean_cols])
dfboolean = df.select([c for c in df.columns if c in boolean_cols])

for boolc in boolean_cols:
  x = dfboolean.select(boolc).distinct().count()
  if x <= 1:
    drop_boolean_cols.append(boolc)
    
for item in drop_boolean_cols:
  if item in dataname:
    dataname.remove(item)
    
print('Total columns (after removing all same value):', len(dataname), ' - dropped ', len(drop_boolean_cols), ' boolean out of ', len(boolean_cols))

# drop all other / array columns.
drop_othernarray_cols = ['author_flair_richtext', 'link_flair_richtext', 'preview']

for item in drop_othernarray_cols:
  if item in dataname:
    dataname.remove(item)

print('total columns (after dropping other/array cols):', len(dataname), ' - dropped all 3')

Total columns: 58
Total columns (after removing null/irrelavent string cols): 36 - dropped 22 string out of 29 (includes permalink, author and title that are used for deriving other features but not used as features themselves)
Total columns (after removing 3 null bigint cols and 2 bigint cols with same values): 31 - dropped 5 bigint out of 8 includes permalink, author and title that are used for deriving other features but not used as features themselves
Total columns (after removing all same value): 19 - dropped 12 boolean out of 18
total columns (after dropping other/array cols): 16 - dropped all 3

In [11]:
# Printing useful columns  - to be used for next step.
# =============================

print('The useful columns are:', dataname)

The useful columns are: ['author', 'author_cakeday', 'brand_safe', 'can_gild', 'created_utc', 'domain', 'is_crosspostable', 'no_follow', 'num_comments', 'over_18', 'permalink', 'score', 'subreddit_id', 'suggested_sort', 'title', 'whitelist_status']